In [169]:
%reload_ext autoreload
%autoreload 2

import polars as pl
import numpy as np
import plotly.express as px
import sys
sys.path.insert(0, "../") 
from _parameters import *
import _functions as f

In [170]:
player_rating = 1500
opponent_rating = 1000
es = f.elo_expected_score(player_rating, opponent_rating, ELO_SCALE)
print(f'expected score = {es}')
print(f'delta rating if player wins = {f.delta_rating(1, es, ELO_K)}')
print(f'delta rating if player loses = {f.delta_rating(0, es, ELO_K)}')


expected score = 0.9467597847979775
delta rating if player wins = 2
delta rating if player loses = -30


In [177]:
skill_list_normal = f.randn_skew_fast(NUMBER_OF_PLAYERS, alpha=0, loc=AVERAGE_SKILL, scale=SIGMA_SKILL)
skill_list_skewed = f.randn_skew_fast(NUMBER_OF_PLAYERS, alpha=SKEW_PARAMETER, loc=AVERAGE_SKILL, scale=SIGMA_SKILL)

fig = px.histogram(
    x=np.concatenate([skill_list_normal, skill_list_skewed]),
    color=np.array(['Normal'] * len(skill_list_normal) + ['Skewed'] * len(skill_list_skewed)),
    nbins=50,
    opacity=0.5,
    labels={'x': 'Skill', 'color': 'Type'},
    title='Skill Distributions: Normal vs Skewed'
)
fig.update_layout(barmode='overlay')
fig.show()

let's say that the player has an "ideal" score. let's see how many games it needs to get a stable score while facing random rating players sampled from a skewed distribution.

In [176]:
IDEAL_PLAYER_RATING = 2000
current_player_rating = ELO_START
number_of_matches= 10000
print_flag = False

print('Parameters of the simulation:')
print(f' - Ideal player rating: {IDEAL_PLAYER_RATING}')
print(f' - Starting player rating: {ELO_START}')
print(f' - Elo scale: {ELO_SCALE}')
print(f' - Elo k-factor: {ELO_K}')
print(f' - Number of matches: {number_of_matches}')
print('-------------------')

player_rating_history = [current_player_rating]

# initialize seed number to get a reproducible sequence of random numbers
np.random.seed(SEED)
for match in range(number_of_matches):
    current_player_rating = player_rating_history[-1]
    opponent_rating = int(np.random.choice(skill_list_skewed, 1))
    es = f.elo_expected_score(current_player_rating, opponent_rating, ELO_SCALE)
    chance_of_winning = f.elo_expected_score(IDEAL_PLAYER_RATING, opponent_rating, ELO_SCALE)
    if np.random.rand() < chance_of_winning:
        # player wins
        actual_score = 1
        delta_rating = f.delta_rating(actual_score, es, ELO_K)
    else:
        # player loses
        actual_score = 0
        delta_rating = f.delta_rating(actual_score, es, ELO_K)
    new_player_rating = current_player_rating + delta_rating
    if print_flag:
        print(f'Match {match+1}')
        print(f'Player current rating: {current_player_rating}')
        print(f'Opponent rating: {opponent_rating}')
        print(f'Expected score: {es:.3f}')
        print(f'Chance of winning: {chance_of_winning:.3f}')
        print(f'Actual score: {actual_score:.3f}')
        print(f'Delta rating: {delta_rating}')
        print(f'New player rating: {new_player_rating}')
        print('-------------------')
    player_rating_history.append(new_player_rating)

fig = px.line(y=player_rating_history, labels={'x': 'Match', 'y': 'Player Rating'}, title='Player Rating Over Matches')
fig.add_hline(y=IDEAL_PLAYER_RATING, line_dash="dash", line_color="red", annotation_text="Ideal Player Rating", annotation_position="top left")
fig.show()


Parameters of the simulation:
 - Ideal player rating: 2000
 - Starting player rating: 1200
 - Elo scale: 400
 - Elo k-factor: 32
 - Number of matches: 10000
-------------------


C:\Users\en33561\AppData\Local\Temp\ipykernel_10692\3454818153.py:20: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

